# ComfortHealth Strategic Plan Optimization – Code Examples

## 1. Demand Forecasting

In [ ]:
growth_rate = (recent_demands.iloc[-1] - recent_demands.iloc[0]) / (recent_years.iloc[-1] - recent_years.iloc[0])
forecasted_demand = last_demand + growth_rate * years_since_last

I implemented a simple linear forecasting model to project regional healthcare demand based on past trends. This gave us a realistic input for the optimization and ensured scalability planning aligned with demographic shifts.

## 2. Objective Function – Maximize Net Profit

In [ ]:
model += pulp.lpSum([
    demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * (
        42 - 37.85 * (1.025 ** (y - 2023)) -
        distances.loc[(distances['origin'] == i) & (distances['destination'] == j), 'distance'].values[0] * 0.25
    ) * w[i, j, y]
    for i in centers_list for j in regions_list for y in forecast_years
]) - pulp.lpSum([
    hiring_cost * H[i, y] + centers.loc[centers['Center'] == i, 'Operating Costs'].values[0] * O[i, y]
    for i in centers_list for y in forecast_years
])

I constructed a detailed profit function combining hourly reimbursements, annual salary increases, distance-based travel costs, hiring expenses, and center operating costs. This allowed us to objectively evaluate the trade-offs between expansion and profitability.

## 3. Workforce Availability Constraint

In [ ]:
model += pulp.lpSum([
    demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * w[i, j, y]
    for j in regions_list
]) <= annual_hp_hours * (initial_hp + pulp.lpSum([H[i, yy] for yy in forecast_years if yy <= y]))

I encoded workforce constraints to ensure no center was assigned more demand than its staff could handle, dynamically accounting for cumulative hiring over time.

## 4. Capacity and Operational Constraints

In [ ]:
model += pulp.lpSum([
    demands.loc[(demands['Region'] == j) & (demands['Year'] == y), 'Demand'].values[0] * w[i, j, y]
    for j in regions_list
]) <= capacity * O[i, y]

This constraint ensured that only open centers could serve patients and never beyond their operational capacity, reinforcing realistic and cost-conscious planning.

## 5. Binary and Integer Decision Variables

In [ ]:
w = pulp.LpVariable.dicts('w', [...], cat='Binary')  # Region allocation
H = pulp.LpVariable.dicts('H', [...], lowBound=0, cat='Integer')  # Number of HPs hired
O = pulp.LpVariable.dicts('O', [...], cat='Binary')  # Center operational status

I structured the problem with binary and integer decision variables, reflecting real-world decisions like whether to open a center, how many people to hire, and which center should serve each region.